### Top Coins

In [ ]:
from curl_cffi import requests
from typing import List
from pydantic import BaseModel
import pandas as pd

params = {
    "start": 1,
    "limit": 100,
    "sortBy": "rank",
    "sortType": "desc",
    "convert": "USD,BTC,ETH",
    "cryptoType": "all",
    "tagType": "all",
    "audited": "false",
    "aux": "cmc_rank,date_added",
    "marketCapRange": "100000000~",
    "volume24hRange": "1000000~",
    "ageRange": "4204800~",
}
r = requests.get(
    "https://api.coinmarketcap.com/data-api/v3/cryptocurrency/listing",
    params=params,
    impersonate="chrome",
)
data = r.json()


class CryptoCurrency(BaseModel):
    id: int
    name: str
    symbol: str
    dateAdded: str


class CryptoData(BaseModel):
    cryptoCurrencyList: List[CryptoCurrency]


class ApiResponse(BaseModel):
    data: CryptoData


parsed = ApiResponse(**data)
df = pd.DataFrame(
    [
        {
            "name": coin.name.lower(),
            "symbol": coin.symbol.upper(),
        }
        for coin in parsed.data.cryptoCurrencyList
    ]
)

df = df[df["symbol"] != "usdt"]
df = df.set_index("name")

In [ ]:
from datetime import date
from binance.download_kline import download_monthly_klines

# 🔹 Build symbols list (exclude USDT itself)
symbols = [f"{sym}USDT" for sym in df["symbol"] if sym.upper() != "USDT"]
# symbols = [df.loc["bitcoin"].symbol + "USDT"]  # remove this to cover all coins

# 🔹 Fixed start and end dates
start_date = date(2017, 1, 1)
end_date = date.today()

print("Downloading from", start_date, "to", end_date)

# 🔹 Loop over each symbol
for symbol in symbols:
    print(f"\n🚀 Starting download for {symbol} ...")
    download_monthly_klines(
        trading_type="spot",
        symbols=[symbol],
        num_symbols=1,
        intervals=["1h"],
        years=range(start_date.year, end_date.year + 1),
        months=range(1, 13),  # downloader filters with start/end anyway
        start_date=start_date.strftime("%Y-%m-%d"),
        end_date=end_date.strftime("%Y-%m-%d"),
        folder="./",
        checksum=0,
    )

print("\n✅ All downloads and completed.")